In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# housing data import and add bias ahead of matrix
housing = fetch_california_housing()
m, n = housing.data.shape

In [2]:
n_epochs = 10000
learning_rate = 0.01

In [3]:
scaled_housing_data = housing.data / housing.data.max(axis=0)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]
#scaled_housing_data_plus_bias

In [4]:
%%time
tf.compat.v1.disable_v2_behavior()
## tensorflow model
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

## random support of theta
theta = tf.Variable(tf.random.normal([n+1, 1], -1.0, 1.0), name="theta")

## random support of theta
y_pred = tf.matmul(X, theta, name="preditions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)

training_op = tf.compat.v1.assign(theta, theta - learning_rate * gradients)

init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            print("Epoch {}, MSE = {}".format(epoch, mse.eval()))
        sess.run(training_op)
        
    best_beta = theta.eval()
    print("best beta : {}".format(best_beta))

Instructions for updating:
non-resource variables are not supported in the long term
Epoch 0, MSE = 30.820446014404297
Epoch 1000, MSE = 0.9694516658782959
Epoch 2000, MSE = 0.8175520896911621
Epoch 3000, MSE = 0.7355639934539795
Epoch 4000, MSE = 0.6912153959274292
Epoch 5000, MSE = 0.6671860218048096
Epoch 6000, MSE = 0.6541329622268677
Epoch 7000, MSE = 0.6470103859901428
Epoch 8000, MSE = 0.6430922150611877
Epoch 9000, MSE = 0.6409059762954712
best beta : [[ 1.3219917 ]
 [ 6.0944114 ]
 [ 0.85237694]
 [-0.24804655]
 [ 0.51530284]
 [-0.20016727]
 [-1.5497631 ]
 [-2.6561642 ]
 [ 0.92432386]]
CPU times: user 6.72 s, sys: 1.34 s, total: 8.07 s
Wall time: 4.8 s
